In [1]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.schema import Document
import markdown_it
import logging
import sys
import pandas as pd
import nest_asyncio
import json
import os
from tqdm import tqdm

nest_asyncio.apply()


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## 设置相关配置  
Ollama需要提前安装，并使用ollama serve启动服务

In [2]:
# 数据源
folder_path = "./data/md"

# 设置LLM
Settings.llm = Ollama(model="qwen2.5-7b:latest", temperature=0.1, context_window=8192, request_timeout=240)
Settings.chunk_size = 2048
Settings.chunk_overlap = 256

## 将markdown加载到llamaindex的document中

In [3]:
def load_markdown_files_from_folder(folder_path):
    """Load all Markdown files from a folder and return their paths.
    Aragruments:
        folder_path (str): Path to the folder containing Markdown files.
    
    Returns:
        list: List of paths to Markdown files.
    
    """
    markdown_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.md'):
                markdown_files.append(os.path.join(root, file))
    return markdown_files


def parse_markdown_file(file_path):
    """Parse a single Markdown file and return a list of TextNodes.
    
    Arguments:
        file_path {str} -- Path to the Markdown file
    
    Returns:
        list -- List of Documents
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()
    
    md = markdown_it.MarkdownIt()
    tokens = md.parse(markdown_content)
    
    documents = []
    current_title = ""
    current_paragraph = ""

    for token in tokens:
        if token.type == 'heading_open':
            # Save the previous paragraph as a Document if it exists
            if current_paragraph.strip():
                doc = Document(
                    text=current_paragraph,
                    metadata={"keyword": current_title}
                )
                documents.append(doc)
            
            # Start a new title
            current_title = ""
            current_paragraph = ""
        
        elif token.type == 'inline':
            if current_title == "":
                current_title += token.content
            else:
                current_paragraph += token.content
        
        elif token.type == 'paragraph_close':
            # End of a paragraph, add it to the current content
            current_paragraph += "\n"
        
        elif token.type == 'heading_close':
            # End of a heading, prepare for next content
            pass
    
    # Add the last paragraph if it exists
    if current_paragraph.strip():
        doc = Document(
            text=current_paragraph,
            metadata={"keyword": current_title}
        )
        documents.append(doc)
    
    return documents


def parse_markdown_folder(folder_path):
    """Parse all Markdown files in a folder and optionally apply semantic splitting.
    Arguments:
        folder_path (str): Path to the folder containing Markdown files.

        Returns:

    Returns:
        list: A list of TextNodes representing the parsed Markdown files.
    
    """
    print(f"Parsing Markdown files from {folder_path}")
    markdown_files = load_markdown_files_from_folder(folder_path)
    all_documents = []


    for index in tqdm(range(len(markdown_files)), desc="Parsing Markdown files"):
        file_path = markdown_files[index]
        print(f"Parsing {file_path}")
        documents = parse_markdown_file(file_path)

        all_documents.extend(documents)
    
    return all_documents

## 生成问题

In [4]:
text_question_template = PromptTemplate(
    """\
下面是上下文信息。
---------------------
{context_str}
---------------------
给定上下文信息而不是先验知识。
仅根据下面的查询生成问题。注意问题必须与舌象描述相关。且问题必须是中文
{query_str}
"""
)

In [5]:
documents = parse_markdown_folder(folder_path)

# instantiate a DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=Settings.llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
    text_question_template=text_question_template,
    show_progress=True,
)

dataset = dataset_generator.generate_dataset_from_nodes()

Parsing Markdown files from ./data/


Parsing Markdown files:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing Markdown files: 100%|██████████| 1/1 [00:00<00:00,  5.49it/s]
/home/sc/anaconda3/envs/tc_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Parsing ./data/Chinese medicine tongue diagnosis.md


  0%|          | 0/287 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


100%|██████████| 1/1 [00:10<00:00, 10.20s/it]


## 保存数据到jsonl文件

In [7]:
data = dataset.to_pandas()

output_file = "./data/json/tongue_dataset.jsonl"


final_results = []
for i, row in data.iterrows():
        write_data = {
            "instruction": row[0],
            "input": "",
            "output": row[2],
        }
        final_results.append(write_data)


with open(output_file, 'w', encoding='utf-8') as output_file:
    json.dump(final_results, output_file, ensure_ascii=False, indent=4)



/tmp/ipykernel_1351786/1290852310.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "instruction": row[0],
/tmp/ipykernel_1351786/1290852310.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "output": row[2],
